# Bir regresyon modeli oluşturma: verileri hazırlama ve görselleştirme

## **Balkabağı için Doğrusal Regresyon - Ders 2**
#### Giriş

Tidymodels ve Tidyverse ile makine öğrenimi modeli oluşturma sürecine başlamak için gerekli araçları kurduğunuza göre, artık verilerinizle ilgili sorular sormaya hazırsınız. Verilerle çalışırken ve ML çözümleri uygularken, veri setinizin potansiyelini doğru bir şekilde açığa çıkarmak için doğru soruyu nasıl soracağınızı anlamak çok önemlidir.

Bu derste öğreneceksiniz:

-   Model oluşturma için verilerinizi nasıl hazırlayacağınızı.

-   Veri görselleştirme için `ggplot2`'yi nasıl kullanacağınızı.

Cevaplanmasını istediğiniz soru, hangi tür ML algoritmalarını kullanacağınızı belirleyecektir. Ve alacağınız cevabın kalitesi, büyük ölçüde verilerinizin doğasına bağlı olacaktır.

Bunu pratik bir alıştırma ile görelim.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>@allison_horst tarafından yapılmış sanat eseri</figcaption>


<!--![@allison_horst tarafından yapılmış sanat eseri](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>@allison_horst tarafından yapılmış sanat eseri-->


## 1. Kabak verilerini içe aktarma ve Tidyverse'i çağırma

Bu dersin verilerini işlemek için aşağıdaki paketlere ihtiyacımız olacak:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/), veri bilimini daha hızlı, kolay ve eğlenceli hale getirmek için tasarlanmış bir [R paketleri koleksiyonudur](https://www.tidyverse.org/packages).

Paketleri şu şekilde yükleyebilirsiniz:

`install.packages(c("tidyverse"))`

Aşağıdaki script, bu modülü tamamlamak için gerekli paketlere sahip olup olmadığınızı kontrol eder ve eksik olanları sizin için yükler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Şimdi, bazı paketleri çalıştıralım ve bu ders için sağlanan [veriyi](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) yükleyelim!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Hızlı bir `glimpse()` işlemi, boşluklar olduğunu ve metin (`chr`) ile sayısal verilerin (`dbl`) karışık olduğunu hemen gösteriyor. `Date` karakter türünde ve ayrıca verilerin `sacks`, `bins` ve diğer değerlerin karışımı olduğu garip bir `Package` adlı sütun da var. Aslında, veri biraz karışık 😤.

Aslında, tamamen kullanıma hazır bir veri setinin size sunulması ve doğrudan bir ML modeli oluşturmak için uygun olması pek yaygın değildir. Ama endişelenmeyin, bu derste, standart R kütüphanelerini kullanarak ham bir veri setini nasıl hazırlayacağınızı öğreneceksiniz 🧑‍🔧. Ayrıca, veriyi görselleştirmek için çeşitli teknikleri de öğreneceksiniz. 📈📊
<br>

> Bir hatırlatma: Pipe operatörü (`%>%`), bir nesneyi ileriye doğru bir fonksiyona veya çağrı ifadesine geçirerek işlemleri mantıksal bir sırayla gerçekleştirir. Pipe operatörünü kodunuzda "ve sonra" demek gibi düşünebilirsiniz.


## 2. Eksik verileri kontrol et

Veri bilimcilerin en sık karşılaştığı sorunlardan biri eksik veya kayıp verilerdir. R, eksik veya bilinmeyen değerleri özel bir işaret değeri olan `NA` (Not Available) ile temsil eder.

Peki, veri çerçevesinde eksik değerler olduğunu nasıl anlayabiliriz?
<br>
-   En basit yöntem, mantıksal nesneler `TRUE` veya `FALSE` döndüren temel R fonksiyonu `anyNA`yı kullanmaktır.


In [ ]:
pumpkins %>% 
  anyNA()

Harika, bazı eksik veriler var gibi görünüyor! Bu başlamak için iyi bir yer.

-   Bir başka yöntem, hangi bireysel sütun elemanlarının eksik olduğunu `TRUE` mantıksal değeriyle gösteren `is.na()` fonksiyonunu kullanmaktır.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Tamam, iş tamamlandı ancak bu kadar büyük bir veri çerçevesiyle, tüm satırları ve sütunları tek tek incelemek verimsiz ve pratikte imkansız olurdu😴.

-   Daha sezgisel bir yöntem, her sütundaki eksik değerlerin toplamını hesaplamak olacaktır:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Çok daha iyi! Eksik veriler var, ancak belki de bu, yapılacak görev için önemli olmayabilir. Bakalım, daha fazla analiz ne ortaya çıkaracak.

> R, harika paketler ve fonksiyonlar setlerinin yanı sıra çok iyi bir dokümantasyona sahiptir. Örneğin, `help(colSums)` veya `?colSums` kullanarak bu fonksiyon hakkında daha fazla bilgi edinebilirsiniz.


## 3. Dplyr: Veri Manipülasyonu için Bir Dilbilgisi

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>@allison_horst tarafından yapılmış bir illüstrasyon</figcaption>


<!--![@allison_horst tarafından yapılmış bir illüstrasyon](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>@allison_horst tarafından yapılmış bir illüstrasyon-->


[`dplyr`](https://dplyr.tidyverse.org/), Tidyverse içinde yer alan bir paket olup, veri manipülasyonu için bir dilbilgisi sunar ve en yaygın veri manipülasyonu sorunlarını çözmenize yardımcı olan tutarlı bir fiil seti sağlar. Bu bölümde, dplyr'ın bazı fiillerini keşfedeceğiz!  
<br>


#### dplyr::select()

`select()` fonksiyonu, `dplyr` paketinde bulunan ve belirli sütunları seçmenize veya hariç tutmanıza yardımcı olan bir işlevdir.

Veri çerçevenizi daha kolay çalışılabilir hale getirmek için, `select()` kullanarak ihtiyacınız olan sütunları tutup diğerlerini kaldırabilirsiniz.

Örneğin, bu alıştırmada analizimiz `Package`, `Low Price`, `High Price` ve `Date` sütunlarını içerecek. Hadi bu sütunları seçelim.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()`, `dplyr` paketinde bulunan ve mevcut sütunları korurken yeni sütunlar oluşturmanıza veya mevcut sütunları değiştirmenize yardımcı olan bir fonksiyondur.

`mutate` fonksiyonunun genel yapısı şu şekildedir:

`data %>%   mutate(yeni_sutun_adi = icerigi)`

`mutate` fonksiyonunu `Date` sütunu üzerinde şu işlemleri yaparak deneyelim:

1. Tarihleri (şu anda karakter türünde) ay formatına dönüştürün (bunlar ABD tarih formatında, yani `MM/DD/YYYY`).

2. Tarihlerden ay bilgisini çıkararak yeni bir sütuna ekleyin.

R dilinde, [lubridate](https://lubridate.tidyverse.org/) paketi Tarih-Zaman verileriyle çalışmayı kolaylaştırır. Bu nedenle, yukarıdaki hedeflere ulaşmak için `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` fonksiyonlarını kullanalım. Date sütununu, sonraki işlemlerde artık ihtiyaç duymayacağımız için kaldırabiliriz.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Şimdi, bir yeni sütun olan `Price` oluşturalım. Bu sütun, bir kabağın ortalama fiyatını temsil ediyor. Şimdi, `Low Price` ve `High Price` sütunlarının ortalamasını alarak yeni Price sütununu dolduralım.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Evet!💪

"Ama dur bir dakika!", tüm veri setini `View(pumpkins)` ile hızlıca gözden geçirdikten sonra diyeceksiniz ki, "Burada bir gariplik var!"🤔

Eğer `Package` sütununa bakarsanız, kabakların birçok farklı şekilde satıldığını göreceksiniz. Bazıları `1 1/9 bushel` ölçüsünde, bazıları `1/2 bushel` ölçüsünde, bazıları kabak başına, bazıları pound başına, ve bazıları da farklı genişliklerde büyük kutular içinde satılıyor.

Hadi bunu doğrulayalım:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Harika!👏

Balkabaklarını tutarlı bir şekilde tartmak oldukça zor görünüyor, bu yüzden onları filtreleyelim. Bunun için `Package` sütununda *bushel* kelimesini içeren balkabaklarını seçip yeni bir veri çerçevesi olan `new_pumpkins` içine koyalım.


#### dplyr::filter() ve stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): Verilerin yalnızca **satırlarının** koşullarınızı sağladığı bir alt kümesini oluşturur, bu durumda `Package` sütununda *bushel* kelimesini içeren kabaklar.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): Bir dizgede bir desenin varlığını veya yokluğunu algılar.

[`stringr`](https://github.com/tidyverse/stringr) paketi, yaygın dizge işlemleri için basit fonksiyonlar sağlar.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Kabakları kile ile içeren yaklaşık 415 satıra kadar daralttığımızı görebilirsiniz.🤩
<br>


#### dplyr::case_when()

**Ama durun! Yapılacak bir şey daha var**

Her satırda buşel miktarının değiştiğini fark ettiniz mi? Fiyatlandırmayı normalize etmeniz gerekiyor, böylece fiyatlandırmayı 1 1/9 veya 1/2 buşel yerine buşel başına gösterebilirsiniz. Şimdi fiyatları standartlaştırmak için biraz matematik yapma zamanı.

Fiyat sütununu bazı koşullara bağlı olarak *değiştirmek* için [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) fonksiyonunu kullanacağız. `case_when`, birden fazla `if_else()` ifadesini vektörleştirmenize olanak tanır.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Şimdi, bushel ölçümüne göre birim fiyat analizini yapabiliriz. Ancak, tüm bu kabak bushel'leri üzerine yapılan çalışma, `verinizin doğasını anlamanın` ne kadar `önemli` olduğunu gösteriyor!

> ✅ [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308)'e göre, bir bushel'in ağırlığı ürün türüne bağlıdır çünkü bu bir hacim ölçüsüdür. "Örneğin, bir domates bushel'i 56 pound ağırlığında olmalı... Yapraklar ve yeşillikler daha az ağırlıkla daha fazla alan kaplar, bu yüzden bir ıspanak bushel'i sadece 20 pound'dur." Oldukça karmaşık bir konu! Bushel'den pound'a dönüşüm yapmaya uğraşmayalım, bunun yerine bushel üzerinden fiyatlandırma yapalım. Ancak, tüm bu kabak bushel'leri üzerine yapılan çalışma, verinizin doğasını anlamanın ne kadar önemli olduğunu gösteriyor!
>
> ✅ Yarım bushel ile satılan kabakların çok pahalı olduğunu fark ettiniz mi? Nedenini çözebilir misiniz? İpucu: Küçük kabaklar büyük olanlardan çok daha pahalıdır, muhtemelen bir bushel'de daha fazla yer kapladıkları için. Büyük, içi boş bir turta kabağının kapladığı kullanılmayan alan nedeniyle bushel başına daha fazla küçük kabak bulunur.


Son olarak, sırf macera olsun diye 💁‍♀️, Haydi Ay sütununu da ilk sıraya, yani `Package` sütununun `öncesine` taşıyalım.

Sütunların yerini değiştirmek için `dplyr::relocate()` kullanılır.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Tebrikler! 👌 Artık yeni regresyon modelinizi oluşturabileceğiniz temiz ve düzenli bir veri kümesine sahipsiniz!  
<br>


## 4. ggplot2 ile Veri Görselleştirme

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Dasani Madipalli tarafından hazırlanan infografik</figcaption>


<!--![Dasani Madipalli tarafından hazırlanan infografik](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Şöyle bir *bilgece* söz vardır:

> "Basit bir grafik, veri analistinin zihnine diğer tüm araçlardan daha fazla bilgi taşımıştır." --- John Tukey

Bir veri bilimcinin rolü, üzerinde çalıştığı verinin kalitesini ve doğasını göstermektir. Bunu yapmak için genellikle verinin farklı yönlerini gösteren ilginç görselleştirmeler, grafikler, çizimler ve tablolar oluştururlar. Bu şekilde, görsel olarak ilişkileri ve aksi takdirde ortaya çıkarması zor olan boşlukları gösterebilirler.

Görselleştirmeler, veriye en uygun makine öğrenimi tekniğini belirlemeye de yardımcı olabilir. Örneğin, bir çizgiye uyar gibi görünen bir dağılım grafiği, verinin doğrusal regresyon çalışması için iyi bir aday olduğunu gösterir.

R, grafik oluşturmak için birkaç sistem sunar, ancak [`ggplot2`](https://ggplot2.tidyverse.org/index.html) en zarif ve en esnek olanlardan biridir. `ggplot2`, grafikleri **bağımsız bileşenleri birleştirerek** oluşturmanıza olanak tanır.

Hadi Price ve Month sütunları için basit bir dağılım grafiğiyle başlayalım.

Bu durumda, [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) ile başlayacağız, bir veri seti ve estetik eşleme ([`aes()`](https://ggplot2.tidyverse.org/reference/aes.html) ile) sağlayacağız, ardından dağılım grafikleri için katmanlar ekleyeceğiz (örneğin [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)).


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Bu faydalı bir grafik mi 🤷? Sizi şaşırtan bir şey var mı?

Pek faydalı değil, çünkü tek yaptığı verilerinizi belirli bir ayda noktalar halinde göstermek.  


### **Nasıl faydalı hale getirebiliriz?**

Grafiklerin faydalı veriler göstermesini sağlamak için genellikle verileri bir şekilde gruplandırmanız gerekir. Örneğin, bizim durumumuzda, her ay için kabakların ortalama fiyatını bulmak, verilerimizdeki temel desenlere dair daha fazla içgörü sağlayacaktır. Bu bizi bir başka **dplyr** özelliğine yönlendiriyor:

#### `dplyr::group_by() %>% summarize()`

R'de gruplandırılmış toplama işlemleri kolayca şu şekilde yapılabilir:

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` analiz birimini tüm veri setinden, ay gibi bireysel gruplara değiştirir.

-   `dplyr::summarize()` her bir gruplama değişkeni için bir sütun ve belirttiğiniz özet istatistikler için bir sütun içeren yeni bir veri çerçevesi oluşturur.

Örneğin, **Month** sütunlarına göre kabakları gruplandırmak ve her ay için **ortalama fiyatı** bulmak için `dplyr::group_by() %>% summarize()` kullanabiliriz.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Kısa ve öz!✨

Aylar gibi kategorik özellikler, bir çubuk grafik kullanılarak daha iyi temsil edilir 📊. Çubuk grafiklerden sorumlu katmanlar `geom_bar()` ve `geom_col()`dur. Daha fazla bilgi için `?geom_bar` komutuna göz atabilirsiniz.

Hadi bir tane oluşturalım!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Bu, daha kullanışlı bir veri görselleştirme! Görünüşe göre en yüksek kabak fiyatları Eylül ve Ekim aylarında gerçekleşiyor. Bu beklentinizi karşılıyor mu? Neden veya neden değil?

İkinci dersi tamamladığınız için tebrikler 👏! Verilerinizi model oluşturma için hazırladınız, ardından görselleştirmeler kullanarak daha fazla içgörü ortaya çıkardınız!



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
